In [1]:
import gym
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint


# todos

## Environment
- [done] get ships from contract
   * np.where + mod 
   * create a column `is_available` in `fleet` with fleet availability   
- [ ]create `contracts_df` from `create_contracts()`.
   * Columns : 
     * `start_port`
     * `ship_type`
     * `end_port`
     * `start_day` =  start_step + (steps needed for pick up == dm / 14 knots) ? probably 10 knots
     * `end_day` = start_day + max_distance in distance_matrix / 10 knots in steps
     * `value_in_$` = values_dict for each ship type
     * `assigned_ship` = [0,1,2] None, ship_1, ship_2 
- [ ] create `can_reach` function that calculates if a ship can make it in time for the contract
   * maybe implement `can_reach` with 10,12,14 knots? 



## Agent
- [ ] Action Space:
   * Choose contract Discrete 5 [0,1,2,3,4] choose zero, one, two, three or all contracts
   * For each contract: 
     * Choose ship Discrete 3 [0,1,2]
     * Choose speed Discrete 3 [10,12,14]
- [ ] 

In [8]:
# load data

ports = pd.read_csv('data/ports_10.csv')
dm = pd.read_csv('data/distance_matrix.csv')
fleet = pd.read_csv('data/fleet.csv')

In [9]:
fleet

,ship_name,ship_type,dwt,cii_threshold,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,0,0,1
1,handy_2,handymax,40000,0,0,0,0,1
2,supra_1,supramax,50000,0,0,0,0,1
3,supra_2,supramax,50000,0,0,0,0,1
4,ultra_1,ultramax,60000,0,0,0,0,1
5,ultra_2,ultramax,60000,0,0,0,0,1
6,pana_1,panamax,70000,0,0,0,0,1
7,pana_2,panamax,70000,0,0,0,0,1
8,kamsar_1,kamsarmax,80000,0,0,0,0,1
9,kamsar_2,kamsarmax,80000,0,0,0,0,1


In [100]:
data_dict = {
    "fleet_path":'data/fleet.csv',
    "ports_path":'data/ports_10.csv',
    "dm_path" : 'data/distance_matrix.csv'
}

In [5]:
ports

,number,name,country,region,code,position,latitude,longitude,location
0,1,Shanghai,China,East Asia,CNSHA,"31° 15.500' N, 121° 32.600' E",31.258333,121.543333,Yangtze Delta
1,2,Singapore,Singapore,Southeast Asia,SGSIN,"1° 15.500' N, 103° 47.900' E",1.258330,103.798333,Singapore Strait
2,3,Xiamen,China,East Asia,CNXMN,"24° 27.100' N, 118° 4.300' E",24.451670,118.071667,Taiwan Strait
3,4,Dalian,China,East Asia,CNDLC,"38° 56.300' N, 121° 39.500' E",38.938330,121.658333,Yellow Sea
4,5,Guangzhou,China,East Asia,CNCAN,"23° 3.900' N, 113° 25.150' E",23.065000,113.419167,Pearl River Delta
5,6,Qingdao,China,East Asia,CNTAO,"36° 5.650' N, 120° 19.000' E",36.094170,120.316667,Yellow Sea
6,7,Busan,South Korea,East Asia,KRPUS,"35° 6.500' N, 129° 3.500' E",35.108330,129.058333,Korean Strait
7,8,Hong Kong,Hong Kong,East Asia,HKHKG,"22° 17.400' N, 114° 10.100' E",22.290000,114.168333,Pearl River Delta
8,9,Tianjin,China,East Asia,CNTXG,"38° 58.400' N, 117° 46.300' E",38.973330,117.771667,Yellow Sea
9,10,Kaohsiung,Taiwan,East Asia,TWKHH,"22° 34.800' N, 120° 18.000' E",22.580000,120.300000,Taiwan Strait


In [3]:
# set fleet at random ports

fleet['current_port'] =  np.random.randint(1,11,fleet.shape[0])
fleet

,vessel_name,vessel_type,dwt,cii_required,cii_attained,current_port,current_speed,is_available
0,handy_1,handymax,40000,0,0,6,0,1
1,handy_2,handymax,40000,0,0,7,0,1
2,supra_1,supramax,50000,0,0,5,0,1
3,supra_2,supramax,50000,0,0,5,0,1
4,ultra_1,ultramax,60000,0,0,10,0,1
5,ultra_2,ultramax,60000,0,0,7,0,1
6,pana_1,panamax,70000,0,0,4,0,1
7,pana_2,panamax,70000,0,0,6,0,1
8,kamsar_1,kamsarmax,80000,0,0,6,0,1
9,kamsar_2,kamsarmax,80000,0,0,1,0,1


In [4]:
# create random contracts
def create_contr(seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c

In [5]:
# contracts_df = start_port, ship_type, end_port, start_day, end_day, value, assigned_ship
# 4*365 = 1460 rows
contracts_df = pd.DataFrame(columns=['start_port','ship_type','end_port','start_day','end_day','value','assigned_ship'],
                            index = range(1,1461))
contracts_df 

,start_port,ship_type,end_port,start_day,end_day,value,assigned_ship
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1456,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1457,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1458,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
c_df = contracts_df.copy()

In [102]:
def populate_contracts_df(df=c_df,step=1):
    """
    A function to populate the contracts df at every environment step
    """
    day = step
    # creating 4 contracts for the specific day
    contrs = create_contr(seed=day)
    # getting locations with non zero contracts
    locs = np.nonzero(contrs)[0]
    # getting the ports of the contract locations (location = port + shiptype)
    prts = np.floor_divide(locs,5)
    # ports start at port 1
    prts = prts + 1
    # getting the contract count for a specific location 
    contract_count_per_loc = contrs[locs]
    # fives array to help with modulo
    fives = np.ones((len(locs)))*5
    # find ship type at port with modulo
    x = np.mod(locs,fives)
    # creating the conditions for each shiptype
    conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
    choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
    # getting the shiptype and converting it to str
    ship_types = np.select(conditions,choices)
    ship_types = ship_types.astype(str)
    #
    b = np.repeat(ship_types,contract_count_per_loc)
    c = np.repeat(prts,contract_count_per_loc)
    c2 = np.char.array(['port_']) + c.astype(str)
    d = np.column_stack((c2,b))
    c_df[step:4*step]
    print(d)
    pass


In [114]:
365*3

1095

In [120]:
x = np.arange(start=0,stop=1095,step=3)
print(x)
print(len(x)) # 365
y = np.delete(x,[0])
y = np.insert()
print(y)
print(len(y)) # 364
print(x)
print(len(x)) # 365

[   0    3    6    9   12   15   18   21   24   27   30   33   36   39
   42   45   48   51   54   57   60   63   66   69   72   75   78   81
   84   87   90   93   96   99  102  105  108  111  114  117  120  123
  126  129  132  135  138  141  144  147  150  153  156  159  162  165
  168  171  174  177  180  183  186  189  192  195  198  201  204  207
  210  213  216  219  222  225  228  231  234  237  240  243  246  249
  252  255  258  261  264  267  270  273  276  279  282  285  288  291
  294  297  300  303  306  309  312  315  318  321  324  327  330  333
  336  339  342  345  348  351  354  357  360  363  366  369  372  375
  378  381  384  387  390  393  396  399  402  405  408  411  414  417
  420  423  426  429  432  435  438  441  444  447  450  453  456  459
  462  465  468  471  474  477  480  483  486  489  492  495  498  501
  504  507  510  513  516  519  522  525  528  531  534  537  540  543
  546  549  552  555  558  561  564  567  570  573  576  579  582  585
  588 

In [112]:
step = 1
lower = 
upper = 4

In [113]:
c_df[step-1:4*step][['start_port','ship_type']]

,start_port,ship_type
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN


In [104]:
populate_contracts_df()

[['port_3' 'supramax']
 ['port_7' 'panamax']
 ['port_10' 'supramax']
 ['port_10' 'panamax']]


In [95]:
sy = np.array([0,1,1,0,0,1,0,1,0,0])
print(f"contracts vector is {sy}")
locs = np.nonzero(sy)[0]
print(f"Non zero contracts locations are {locs}")
prts = np.floor_divide(locs,5)
print(f"Contract locations are ports {prts+1}")
contract_count_per_loc = sy[locs]
print(f"Number of contracts for each contract location are {contract_count_per_loc}")
fives = np.ones((len(locs)))*5
# find ship type at port with mode
x = np.mod(locs,fives)
#print(f"Dissered ship type code is {x}")
conditions = [ x == 0 , x == 1, x == 2, x == 3, x == 4 ]
choices = ['handymax','supramax','ultramax','panamax','kamsarmax']
ship_types = np.select(conditions,choices)
ship_types = ship_types.astype(str)
print(f"Dissered ship types are {ship_types}")


# desired output 
# port_1, supra
# port_1, supra
# port_2, pana
# port_2, pana


contracts vector is [0 1 1 0 0 1 0 1 0 0]
Non zero contracts locations are [1 2 5 7]
Contract locations are ports [1 1 2 2]
Number of contracts for each contract location are [1 1 1 1]
Dissered ship types are ['supramax' 'ultramax' 'handymax' 'ultramax']


[['port_1' 'supramax']
 ['port_1' 'ultramax']
 ['port_2' 'handymax']
 ['port_2' 'ultramax']]


In [105]:
def get_ship_type(contrs):
    pass


In [117]:
import gym 
import pandas as pd
import numpy as np
from gym import spaces
from utils.utils import cii_expected

class CarbonEnv(gym.Env):
    """
    Description :

        A custom openai gym environment for the carbon emission problem.

    """


    def __init__(self,data_dict):
        super().__init__()

        NUM_CONTRACTS_PER_STEP = 4
        NUM_SHIPS = 10
        SET_OF_SPEEDS = [10,12,14]
        MIN_REVENUE = 1_000_000
        TERMINATION_FROM_CII = # AT LEAST 5 SHIPS DROP THEIR RATING

        self.fleet = pd.read_csv(data_dict['fleet_path'])
        self.ports = pd.read_csv(data_dict['ports_path'])
        self.dm = pd.read_csv(data_dict['dm_path'])

        # calculate fleet's required cii
        self.fleet['cii_required'] = self.fleet['dwt'].map(cii_expected)

        # set fleet at random ports
        self.fleet['current_port'] = np.random.randint(1,11,self.fleet.shape[0])

        self.state = None
        self.previous_action = None



        observation_space = spaces.Dict({
            #"contracts" : spaces.Box(low=0,high=4,shape=(50,), dtype=np.int32),
            #"speeds" : spaces.Box(low=0,high=2,shape=(10,), dtype=np.int32)
            "daily_contracts" : spaces.Box(low=1,high=4,shape=(4,),dtype=np.int32),
            "available_ships" : spaces.Box(low=0,high=1,shape=(10,),dtype=np.int32)
            })


        action_space = spaces.Dict({
            "choose_contract" : spaces.Box(low=0,high=1,shape=(4,),dtype=np.int32), # (0,0,1,1) deny contracts 1,2 accept contracts 3,4
            "choose_ship" : spaces.Box(low=0, high=10, shape=(4,),dtype=np.int32), # (0,0,5,7) assign ships 5,7 to contracts 3,4
            "choose_speed" : spaces.Box(low=0, high=3, shape=(4,),dtype=np.int32) # (0,0,1,2) 10 knots for ship 5, 12knots for ship 7  
            })






    def step(self, action):
        # Execute one time step in the environment based on the action chosen by the model
        # action will have shape 3x4







        return obs, reward, done, info

    def reset(self):
        # Set the state of the environment to an initial state
        self.state = # empty dataframe with state info
        self.previous_action = None
        return self.state
        


    def render(self, mode='human', close=False):
        # Render the environment to the screen
        pass

    
    def create_contracts(self, seed=None):
        """
        A method that creates 4 random cargo contracts at each env step. \ 
        There are 5 cargo contract types handy,supra,ultra,pana,kamsar for each port.\ 
        The function :
        * returns a 1d array of 50 elements
        * all array elements are integers from 0 to 4
        * the sum of the array must be 4
        """
        contract_count = 4
        np.random.seed(seed)
        x = np.random.multinomial(contract_count, [1/5]*5, None)
        z = np.zeros(shape=45, dtype=np.int32)
        c = np.concatenate((x,z))
        np.random.shuffle(c)
        return c



    pass




In [118]:
envo = CarbonEnv(data_dict)

In [37]:
envo.observation_space['contracts']

Box(0, 4, (50,), int32)

In [41]:
envo.reset()

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0])

In [ ]:
# training loop schema

agent = GraphAttentionModel


years = 3000 
for i in range(years): 
    obs = env.reset()
    done = False 
    score = 0
    day = 1
    while not done: # while year is not done
        act = agent.choose_action(obs)
        new_state, reward, done, info = env.step(act)
        agent.remember(obs,act, reward, new_state, int(done))
        agent.learn()
        score += reward
        day += 1
        obs = new_state

